In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

DATA_DIR    = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"
PLOTS_DIR   = PROJECT_ROOT / "plots"

RESULTS_DIR.mkdir(exist_ok=True)
PLOTS_DIR.mkdir(exist_ok=True)


In [4]:
df = pd.read_csv(
    DATA_DIR / "nifty_with_regimes.csv",
    parse_dates=["timestamp"]
).sort_values("timestamp").reset_index(drop=True)

df[["timestamp", "spot_close", "ema_5", "ema_15", "regime"]].head()


,timestamp,spot_close,ema_5,ema_15,regime
0,2025-01-16 09:15:00,18016.461061,18016.461061,18016.461061,0
1,2025-01-16 09:20:00,18012.338002,18015.086708,18015.945678,1
2,2025-01-16 09:25:00,18033.710411,18021.294609,18018.166270,1
3,2025-01-16 09:30:00,18083.578491,18042.055903,18026.342798,1
4,2025-01-16 09:35:00,18076.319824,18053.477210,18032.589926,1


#### generating EMA crossover signals

In [5]:
df["ema_signal"] = 0

# bullish crossover
df.loc[
    (df["ema_5"] > df["ema_15"]) &
    (df["ema_5"].shift() <= df["ema_15"].shift()),
    "ema_signal"
] = 1

# bearish crossover
df.loc[
    (df["ema_5"] < df["ema_15"]) &
    (df["ema_5"].shift() >= df["ema_15"].shift()),
    "ema_signal"
] = -1


#### applying regime filter

In [6]:
df["trade_signal"] = 0

# LONG only in +1 regime
df.loc[
    (df["ema_signal"] == 1) & (df["regime"] == 1),
    "trade_signal"
] = 1

# SHORT only in -1 regime
df.loc[
    (df["ema_signal"] == -1) & (df["regime"] == -1),
    "trade_signal"
] = -1


#### backtesting engine

In [7]:
trades = []
position = 0
entry_price = None
entry_time = None

for i in range(1, len(df)):
    signal = df.loc[i, "trade_signal"]
    price  = df.loc[i, "spot_open"] if "spot_open" in df.columns else df.loc[i, "spot_close"]
    time   = df.loc[i, "timestamp"]

    # ENTRY
    if position == 0 and signal != 0:
        position = signal
        entry_price = price
        entry_time = time

    # EXIT
    elif position != 0:
        exit_condition = (
            (position == 1 and df.loc[i, "ema_signal"] == -1) or
            (position == -1 and df.loc[i, "ema_signal"] == 1)
        )

        if exit_condition:
            pnl = (price - entry_price) * position
            trades.append({
                "entry_time": entry_time,
                "exit_time": time,
                "position": position,
                "entry_price": entry_price,
                "exit_price": price,
                "pnl": pnl,
                "duration": (time - entry_time).total_seconds() / 60
            })
            position = 0


In [8]:
trades_df = pd.DataFrame(trades)
trades_df.head()


,entry_time,exit_time,position,entry_price,exit_price,pnl,duration
0,2025-01-16 09:25:00,2025-01-16 10:25:00,1,18033.710411,18010.432405,-23.278006,60.0
1,2025-01-16 14:15:00,2025-01-16 14:25:00,1,17708.177499,17667.750099,-40.427400,10.0
2,2025-01-16 14:50:00,2025-01-17 09:35:00,1,17730.180437,17703.517819,-26.662619,1125.0
3,2025-01-17 09:50:00,2025-01-17 09:55:00,1,17746.591906,17721.138698,-25.453208,5.0
4,2025-01-17 10:20:00,2025-01-17 10:50:00,1,17746.083254,17690.177135,-55.906119,30.0


In [9]:
# train test splits

split_time = df["timestamp"].iloc[int(len(df) * 0.7)]

train_trades = trades_df[trades_df["entry_time"] <= split_time]
test_trades  = trades_df[trades_df["entry_time"] > split_time]


#### Performance Matrics

In [10]:
def compute_metrics(trades):
    returns = trades["pnl"]

    total_return = returns.sum()
    win_rate = (returns > 0).mean()
    profit_factor = returns[returns > 0].sum() / abs(returns[returns < 0].sum())
    
    sharpe = returns.mean() / returns.std() * np.sqrt(252 * 75)  # ~75 5-min bars/day
    sortino = returns.mean() / returns[returns < 0].std() * np.sqrt(252 * 75)

    equity = returns.cumsum()
    drawdown = equity - equity.cummax()
    max_dd = drawdown.min()
    calmar = total_return / abs(max_dd)

    return {
        "Total Return": total_return,
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Calmar": calmar,
        "Max Drawdown": max_dd,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Total Trades": len(trades),
        "Avg Trade Duration (min)": trades["duration"].mean()
    }

In [11]:
metrics_train = compute_metrics(train_trades)
metrics_test  = compute_metrics(test_trades)

metrics_df = pd.DataFrame([metrics_train, metrics_test], index=["Train", "Test"])
metrics_df

,Total Return,Sharpe,Sortino,Calmar,Max Drawdown,Win Rate,Profit Factor,Total Trades,Avg Trade Duration (min)
Train,1647.770001,3.569101,14.136755,0.410351,-4015.509445,0.287982,1.081789,441,457.086168
Test,5083.458836,19.749279,110.524998,6.448380,-788.331128,0.337209,1.545897,172,422.848837


In [12]:
trades_df.to_csv(RESULTS_DIR / "baseline_trades.csv", index=False)
metrics_df.to_csv(RESULTS_DIR / "baseline_metrics.csv")

print("Baseline trades & metrics saved")



Baseline trades & metrics saved


#### Equity Curve plot

In [13]:
plt.figure(figsize=(12,5))
trades_df["pnl"].cumsum().plot()
plt.title("Baseline Strategy Equity Curve")
plt.ylabel("PnL")
plt.grid()

plt.savefig(PLOTS_DIR / "equity_curve_baseline.png", dpi=150, bbox_inches="tight")
plt.close()


#### Drawdown plot

In [14]:
equity = trades_df["pnl"].cumsum()
drawdown = equity - equity.cummax()

plt.figure(figsize=(12,4))
drawdown.plot(color="red")
plt.title("Baseline Strategy Drawdown")
plt.grid()

plt.savefig(PLOTS_DIR / "drawdown_baseline.png", dpi=150, bbox_inches="tight")
plt.close()
